# Seventh Data Preparation Process

In this notebook file, the dataset provided by the customer will be eddited for general purposes.

Output features will be examined and processed.

In [1]:
#Libraries are being imported
import math
import pandas as pd

In [2]:
#Library version is being printed
print('pandas Version: ' + pd.__version__)

pandas Version: 1.5.2


In [3]:
#A pandas dataframe named as dataFrame is being created by reading the data pkl file created last process
dataFrame = pd.read_pickle("../Data/WithRelationalTables/FabricWasteFabricDetail.pkl")
dataFrame.head()

,ColorID,ProductDetailID,FabricCode,IsManualProcess,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,Quantity,Repair,Delivery,SecondQuality
0,18,0,2SP050,False,125,150,4,35,3,375,405,20,399,6
1,19,0,2SP050,False,125,150,4,65,3,375,414,22,411,3
2,6,0,2SP050,False,125,150,4,65,3,375,389,19,378,11
3,20,0,2SP050,False,125,150,4,65,3,375,414,18,406,8
4,2,0,2SP050,False,125,150,4,65,3,375,414,16,411,3


In [4]:
#Information of dataframe is being printed
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ColorID          186 non-null    int64 
 1   ProductDetailID  186 non-null    int64 
 2   FabricCode       186 non-null    object
 3   IsManualProcess  186 non-null    bool  
 4   Weight           186 non-null    int64 
 5   Width            186 non-null    int64 
 6   ProductPart      186 non-null    int64 
 7   NotchLayer       186 non-null    int64 
 8   NotchQuantity    186 non-null    int64 
 9   OrderQuantity    186 non-null    int64 
 10  Quantity         186 non-null    int64 
 11  Repair           186 non-null    int64 
 12  Delivery         186 non-null    int64 
 13  SecondQuality    186 non-null    int64 
dtypes: bool(1), int64(12), object(1)
memory usage: 19.2+ KB


In [5]:
#Statistical information of dataframe is being printed
dataFrame.describe()

,ColorID,ProductDetailID,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,Quantity,Repair,Delivery,SecondQuality
count,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000
mean,14.193548,5.580645,165.241935,159.435484,4.801075,44.483871,2.467742,294.489247,317.209677,6.387097,309.295699,7.913978
std,9.653184,6.526988,36.999935,15.936653,2.289388,21.042615,0.792896,113.160409,123.932912,4.987622,122.519425,6.432445
min,0.000000,0.000000,125.000000,120.000000,2.000000,0.000000,0.000000,100.000000,110.000000,0.000000,110.000000,0.000000
25%,4.250000,0.000000,125.000000,150.000000,4.000000,31.000000,2.000000,206.250000,224.000000,3.000000,217.000000,4.000000
50%,14.000000,1.000000,172.500000,160.000000,4.000000,39.000000,3.000000,250.000000,272.000000,6.000000,265.000000,6.000000
75%,22.000000,12.000000,190.000000,171.500000,5.000000,58.000000,3.000000,380.000000,414.000000,8.000000,407.500000,10.000000
max,32.000000,20.000000,250.000000,180.000000,14.000000,111.000000,4.000000,640.000000,698.000000,30.000000,690.000000,38.000000


In [6]:
#Defect quantity is being calculated by summation of Repair and SecondQuality features
#Note: Since SecondQuality data is not reliable, Defect data calculated using these data is also far from reliable
#See Preparation02.ipynb for more information
dataFrame.insert(loc=11, column='Defect', value = dataFrame['Repair'] + dataFrame['SecondQuality'])
dataFrame.head()

,ColorID,ProductDetailID,FabricCode,IsManualProcess,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,Quantity,Defect,Repair,Delivery,SecondQuality
0,18,0,2SP050,False,125,150,4,35,3,375,405,26,20,399,6
1,19,0,2SP050,False,125,150,4,65,3,375,414,25,22,411,3
2,6,0,2SP050,False,125,150,4,65,3,375,389,30,19,378,11
3,20,0,2SP050,False,125,150,4,65,3,375,414,26,18,406,8
4,2,0,2SP050,False,125,150,4,65,3,375,414,19,16,411,3


In [7]:
#Within the scope of this work it is desired to predict 
#at least how many products should be produced for the orders together with the wastage

#If the situation is examined through the example in the first row:
#405 products were produced, 26 of these products were defective products,
#20 of them were repaired successfully but 6 of them were reserved as SecondQuality
#Ordered: 375, Delivered: 399
#The aim of this work is to bring the number of delivered products as close as possible to the number of ordered products

#For this process the DefectRate and SecondQualityRate should be calculated
#The DefectRate can be used for less risky predictions, while the SecondQualityRate can be used for more risky predictions

#RepairRate is being calculated for each row
dataFrame['DefectRate'] = dataFrame.apply(lambda row : row['Defect'] / row['Quantity'], axis = 1)
dataFrame.head()

,ColorID,ProductDetailID,FabricCode,IsManualProcess,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,Quantity,Defect,Repair,Delivery,SecondQuality,DefectRate
0,18,0,2SP050,False,125,150,4,35,3,375,405,26,20,399,6,0.064198
1,19,0,2SP050,False,125,150,4,65,3,375,414,25,22,411,3,0.060386
2,6,0,2SP050,False,125,150,4,65,3,375,389,30,19,378,11,0.077121
3,20,0,2SP050,False,125,150,4,65,3,375,414,26,18,406,8,0.062802
4,2,0,2SP050,False,125,150,4,65,3,375,414,19,16,411,3,0.045894


In [8]:
#SecondQualityRate is being calculated for each row
dataFrame['SecondQualityRate'] = dataFrame.apply(lambda row : row['SecondQuality'] / row['Quantity'], axis = 1)
dataFrame.head()

,ColorID,ProductDetailID,FabricCode,IsManualProcess,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,Quantity,Defect,Repair,Delivery,SecondQuality,DefectRate,SecondQualityRate
0,18,0,2SP050,False,125,150,4,35,3,375,405,26,20,399,6,0.064198,0.014815
1,19,0,2SP050,False,125,150,4,65,3,375,414,25,22,411,3,0.060386,0.007246
2,6,0,2SP050,False,125,150,4,65,3,375,389,30,19,378,11,0.077121,0.028278
3,20,0,2SP050,False,125,150,4,65,3,375,414,26,18,406,8,0.062802,0.019324
4,2,0,2SP050,False,125,150,4,65,3,375,414,19,16,411,3,0.045894,0.007246


In [9]:
#The Delivery column is an integer type attribute that contains
#the number of products that has quality which can be delivered to the customer
#The Delivery feature can be calculated by subtracting the SecondQuality from Quantity
#and since prediction can be done on SecondQuality Feature
#Delivery is being removed from the dataset
dataFrame.drop('Delivery', axis = 1, inplace = True)
dataFrame.head()

,ColorID,ProductDetailID,FabricCode,IsManualProcess,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,Quantity,Defect,Repair,SecondQuality,DefectRate,SecondQualityRate
0,18,0,2SP050,False,125,150,4,35,3,375,405,26,20,6,0.064198,0.014815
1,19,0,2SP050,False,125,150,4,65,3,375,414,25,22,3,0.060386,0.007246
2,6,0,2SP050,False,125,150,4,65,3,375,389,30,19,11,0.077121,0.028278
3,20,0,2SP050,False,125,150,4,65,3,375,414,26,18,8,0.062802,0.019324
4,2,0,2SP050,False,125,150,4,65,3,375,414,19,16,3,0.045894,0.007246


In [10]:
#Since required rates have been calculated, Quantity, Defect, Repair and SecondQuality columns can be removed now
dataFrame.drop('Quantity', axis = 1, inplace = True)
dataFrame.drop('Defect', axis = 1, inplace = True)
dataFrame.drop('Repair', axis = 1, inplace = True)
dataFrame.drop('SecondQuality', axis = 1, inplace = True)
dataFrame.head()

,ColorID,ProductDetailID,FabricCode,IsManualProcess,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,DefectRate,SecondQualityRate
0,18,0,2SP050,False,125,150,4,35,3,375,0.064198,0.014815
1,19,0,2SP050,False,125,150,4,65,3,375,0.060386,0.007246
2,6,0,2SP050,False,125,150,4,65,3,375,0.077121,0.028278
3,20,0,2SP050,False,125,150,4,65,3,375,0.062802,0.019324
4,2,0,2SP050,False,125,150,4,65,3,375,0.045894,0.007246


In [11]:
#New Defect features are being calculated according to the number of orders
#The ceil function is used to round the result to upper integer
dataFrame['Defect'] = dataFrame.apply(lambda row : math.ceil(row['OrderQuantity'] * row['DefectRate']), axis = 1)
dataFrame.head()

,ColorID,ProductDetailID,FabricCode,IsManualProcess,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,DefectRate,SecondQualityRate,Defect
0,18,0,2SP050,False,125,150,4,35,3,375,0.064198,0.014815,25
1,19,0,2SP050,False,125,150,4,65,3,375,0.060386,0.007246,23
2,6,0,2SP050,False,125,150,4,65,3,375,0.077121,0.028278,29
3,20,0,2SP050,False,125,150,4,65,3,375,0.062802,0.019324,24
4,2,0,2SP050,False,125,150,4,65,3,375,0.045894,0.007246,18


In [12]:
#New SecondQuality features are being calculated according to the number of orders
#The ceil function is used to round the result to upper integer
dataFrame['SecondQuality'] = dataFrame.apply(lambda row : math.ceil(row['OrderQuantity'] * row['SecondQualityRate']), axis = 1)
dataFrame.head()

,ColorID,ProductDetailID,FabricCode,IsManualProcess,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,DefectRate,SecondQualityRate,Defect,SecondQuality
0,18,0,2SP050,False,125,150,4,35,3,375,0.064198,0.014815,25,6
1,19,0,2SP050,False,125,150,4,65,3,375,0.060386,0.007246,23,3
2,6,0,2SP050,False,125,150,4,65,3,375,0.077121,0.028278,29,11
3,20,0,2SP050,False,125,150,4,65,3,375,0.062802,0.019324,24,8
4,2,0,2SP050,False,125,150,4,65,3,375,0.045894,0.007246,18,3


In [13]:
#Information of dataframe is being printed
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ColorID            186 non-null    int64  
 1   ProductDetailID    186 non-null    int64  
 2   FabricCode         186 non-null    object 
 3   IsManualProcess    186 non-null    bool   
 4   Weight             186 non-null    int64  
 5   Width              186 non-null    int64  
 6   ProductPart        186 non-null    int64  
 7   NotchLayer         186 non-null    int64  
 8   NotchQuantity      186 non-null    int64  
 9   OrderQuantity      186 non-null    int64  
 10  DefectRate         186 non-null    float64
 11  SecondQualityRate  186 non-null    float64
 12  Defect             186 non-null    int64  
 13  SecondQuality      186 non-null    int64  
dtypes: bool(1), float64(2), int64(10), object(1)
memory usage: 19.2+ KB


In [14]:
#Statistical information of dataframe is being printed
dataFrame.describe()

,ColorID,ProductDetailID,Weight,Width,ProductPart,NotchLayer,NotchQuantity,OrderQuantity,DefectRate,SecondQualityRate,Defect,SecondQuality
count,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000,186.000000
mean,14.193548,5.580645,165.241935,159.435484,4.801075,44.483871,2.467742,294.489247,0.047758,0.026811,13.908602,7.919355
std,9.653184,6.526988,36.999935,15.936653,2.289388,21.042615,0.792896,113.160409,0.028010,0.023099,8.328864,6.445527
min,0.000000,0.000000,125.000000,120.000000,2.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000
25%,4.250000,0.000000,125.000000,150.000000,4.000000,31.000000,2.000000,206.250000,0.028916,0.011349,8.000000,4.000000
50%,14.000000,1.000000,172.500000,160.000000,4.000000,39.000000,3.000000,250.000000,0.041523,0.020425,12.000000,6.000000
75%,22.000000,12.000000,190.000000,171.500000,5.000000,58.000000,3.000000,380.000000,0.062802,0.035610,17.750000,10.000000
max,32.000000,20.000000,250.000000,180.000000,14.000000,111.000000,4.000000,640.000000,0.165803,0.165803,60.000000,42.000000


In [15]:
#dataFrame is being saved as pkl files
dataFrame.to_pickle("../Data/FinalPreparation/FabricWaste.pkl")